
Fail Fast, with:
  - General Function Inputs
    - guard clauses
    - checked type annotations
  - Data Structures
    - Pydantic
    - Pandera
    - PathSchema (https://github.com/Apollo-Roboto/python-pathschema)
  - CLI Inputs
    - Argparse
    - Click


In [240]:
from typing import Any, Type

def check(code: str, expected: Any | Type[Exception], exception_message: str = "", verbose = True):
    """
    a "pytest-lite" function.
    
    Takes code to evaluate and what's expected (whether a value, a exception type, or addtionally even a substring in the exception message).
    Returns whether the exception was met, and prints a message describing the finding.
    """
    try:
        output = eval(code) 

    except BaseException as exc:
        output = exc
        if type(expected) == type and issubclass(expected, BaseException):
            valid_exception_type = isinstance(exc, expected)
            valid_exception_message = exception_message in str(exc)
            valid = valid_exception_type and valid_exception_message
            if not valid:
                if not valid_exception_type:
                    expected_str = expected.__name__
                else:
                    expected_str = '\"...' + exception_message + '...\"'
            else:
                expected_str = ''
        else:
            valid = False
            expected_str = str(expected)

        output_str = type(output).__name__
    
    else:
        if type(expected) == type and issubclass(expected, Exception):
            valid = False
            expected_str = expected.__name__
        elif type(expected) == type:
            valid = True
            expected_str = expected.__name__ 
            
        else:
            valid = output == expected
            expected_str = str(expected)

        if " object at " in str(output):
            output_str = type(output).__name__
        else:
            output_str = str(output)

    
    if verbose:
        valid_str = "✅" if valid else "❌"

        # output_str = output if not isinstance(output, Exception) else type(output).__name__
        print(valid_str, code, "->", output_str, "" if valid else f"(Expected: {expected_str})")
    return valid
        


## Type Annotations

### Numbers

In [103]:
from typing import Union

# from strictest to least-strict
from builtins import int, float
from typing import SupportsFloat, SupportsInt # Can have "float()" or "int()" called on it
from numbers import Number, Real  # General, nonspecific numbers (works also with numpy values)
from typing import Any  # the "who cares?" type
import numpy as np

Scalar = Union[int, float, np.generic]

**Example**

In [155]:
def greet(name):
    """
    Says Hi to whoever you want!
    """

    ## Guard Clauses Go Here: ######
    if isinstance(name, (float, int)):
        raise TypeError("`name` should be a string. You are not a number.")
    ##################################
    
    return f"Hi, {name}!"


check("greet('Nicholas')", "Hi, Nicholas!");
check("greet(24601)", TypeError, "You are not a number");

✅ greet('Nicholas') -> Hi, Nicholas! 
✅ greet(24601) -> TypeError 


**Exercise**

In [237]:
def total_length(x, y):
    """
    Computes the total of two lengths of wire.

    Arguments:
      - x: a positive number
      - y: another positive number

    """

    ## Guard Clauses Go Here: ####


    ##############################

    return x + y


check("total_length(3.2, 1.2)", 4.4)
check("total_length([1, 2], [])", TypeError, "number")
check("total_length(-3, 5)", ValueError, "positive")
check("total_length(3, -5.2)", ValueError, "positive")
check("total_length(3, 'a')", TypeError, "number")
check("total_length('hello, ', 'world')", TypeError, "number")
check("total_length(1., 2)", 3.)
check("total_length(np.float32(3), 3)", np.float32(6));

✅ total_length(3.2, 1.2) -> 4.4 
❌ total_length([1, 2], []) -> [1, 2] (Expected: TypeError)
❌ total_length(-3, 5) -> 2 (Expected: ValueError)
❌ total_length(3, -5.2) -> -2.2 (Expected: ValueError)
❌ total_length(3, 'a') -> TypeError (Expected: "...number...")
❌ total_length('hello, ', 'world') -> hello, world (Expected: TypeError)
✅ total_length(1., 2) -> 3.0 
✅ total_length(np.float32(3), 3) -> 6.0 


In [236]:
def total_length(x, y):
    """
    Computes the total of two lengths of wire.

    Arguments:
      - x: a positive number
      - y: another positive number

    """

    ## Guard Clauses Go Here: ######################
    if not isinstance(x, Real):
        raise TypeError("x should be number.")
    if x < 0:
        raise ValueError("x should be positive.")
    if not isinstance(y, Real):
        raise TypeError("y should be a number.")
    if y < 0:
        raise ValueError("y should be positive.")
    ################################################
    
    return x + y


check("total_length(3.2, 1.2)", 4.4)
check("total_length([1, 2], [])", TypeError, "number")
check("total_length(-3, 5)", ValueError, "positive")
check("total_length(3, -5.2)", ValueError, "positive")
check("total_length(3, 'a')", TypeError, "number")
check("total_length('hello, ', 'world')", TypeError, "number")
check("total_length(1., 2)", 3.)
check("total_length(np.float32(3), 3)", np.float32(6));

✅ total_length(3.2, 1.2) -> 4.4 
✅ total_length([1, 2], []) -> TypeError 
✅ total_length(-3, 5) -> ValueError 
✅ total_length(3, -5.2) -> ValueError 
✅ total_length(3, 'a') -> TypeError 
✅ total_length('hello, ', 'world') -> TypeError 
✅ total_length(1., 2) -> 3.0 
✅ total_length(np.float32(3), 3) -> 6.0 


**Exercise**

In [206]:

def translate(rna):
    """
    Change a DNA sequence into an RNA sequence.
    """

    ## Guard Clauses Go Here: ##################



    ############################################

    from urllib.request import urlopen
    import json

    codons_url = "https://raw.githubusercontent.com/nickdelgrosso/dna-transcription-kata/refs/heads/master/data/codons.json"
    with urlopen(codons_url) as response_c:
        peptides = json.loads(response_c.read())

    peptides_url = "https://raw.githubusercontent.com/nickdelgrosso/dna-transcription-kata/refs/heads/master/data/peptides.json"
    with urlopen(peptides_url) as response_p:
        peptides_shorts = json.loads(response_p.read())
    
    
    out = []
    for c0, c1, c2 in zip(rna[::3], rna[1::3], rna[2::3]):
        codon = (c0 + c1 + c2)
        peptide = peptides[codon]
        peptide_short = peptides_shorts[peptide.lower()]
        out.append(peptide_short)

    return "".join(out)
    

check("translate('CCC')", 'P');
check("translate('GCAUUA')", 'AL');
check("translate('gca')", ValueError, "upper")
check("translate('TTT')", ValueError, "GCAU")
check("translate('GG')", ValueError, "three")
# check("")


✅ translate('CCC') -> P 
✅ translate('GCAUUA') -> AL 
❌ translate('gca') -> KeyError (Expected: ValueError)
❌ translate('TTT') -> KeyError (Expected: ValueError)
❌ translate('GG') ->  (Expected: ValueError)


False

In [207]:

def translate(rna):
    """
    Change a DNA sequence into an RNA sequence.
    """

    ## Guard Clauses Go Here: ##################
    if not rna.isupper():
        raise ValueError("nucleotides should be upper-case letters.")
    
    if 'T' in rna:
        raise ValueError("only GCAU allowed")
    
    if len(rna) % 3:
        raise ValueError("length of rna sequence should be a multiple of three")
    

    ############################################

    from urllib.request import urlopen
    import json

    codons_url = "https://raw.githubusercontent.com/nickdelgrosso/dna-transcription-kata/refs/heads/master/data/codons.json"
    with urlopen(codons_url) as response_c:
        peptides = json.loads(response_c.read())

    peptides_url = "https://raw.githubusercontent.com/nickdelgrosso/dna-transcription-kata/refs/heads/master/data/peptides.json"
    with urlopen(peptides_url) as response_p:
        peptides_shorts = json.loads(response_p.read())
    
    
    out = []
    for c0, c1, c2 in zip(rna[::3], rna[1::3], rna[2::3]):
        codon = (c0 + c1 + c2)
        peptide = peptides[codon]
        peptide_short = peptides_shorts[peptide.lower()]
        out.append(peptide_short)

    return "".join(out)
    

check("translate('CCC')", 'P');
check("translate('GCAUUA')", 'AL');
check("translate('gca')", ValueError, "upper")
check("translate('TTT')", ValueError, "GCAU")
check("translate('GG')", ValueError, "three")


✅ translate('CCC') -> P 
✅ translate('GCAUUA') -> AL 
✅ translate('gca') -> ValueError 
✅ translate('TTT') -> ValueError 
✅ translate('GG') -> ValueError 


True

## Data Validation should also be done when creating instances

In [250]:
class Person:

    def __init__(self, name, age) -> None:

        self.name = name
        self.age = age

        ## Guard Clauses Go Here: ##########

        if not isinstance(self.age, (int,)):
            raise TypeError("Age should be an integer.")
        
        if self.age < 0:
            raise ValueError("Age should be positive.")
        
        if not self.name:
            raise ValueError("Name should not be empty.")

        ####################################



check("Person('Nick', 37)", Person)
check("Person('Santa', 'old')", TypeError, "integer")
check("Person('', -200)", ValueError, "positive")
check("Person('', 12)", ValueError, "empty")

✅ Person('Nick', 37) -> Person 
✅ Person('Santa', 'old') -> TypeError 
✅ Person('', -200) -> ValueError 
✅ Person('', 12) -> ValueError 


True

## Dataclasses Have a Simpler Syntax, and they make it clearer where "post-init" checks are

In [252]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int

    def __post_init__(self):
        ...
        ## Guard Clauses Go Here: #########
        if not isinstance(self.age, (int,)):
            raise TypeError("Age should be an integer.")
        
        if self.age < 0:
            raise ValueError("Age should be positive.")
        
        if not self.name:
            raise ValueError("Name should not be empty.")

        ###################################


check("Person('Nick', 37)", Person)
check("Person('Santa', 'old')", TypeError, "integer")
check("Person('', -200)", ValueError, "positive")
check("Person('', 12)", ValueError, "empty")

✅ Person('Nick', 37) -> Person(name='Nick', age=37) 
✅ Person('Santa', 'old') -> TypeError 
✅ Person('', -200) -> ValueError 
✅ Person('', 12) -> ValueError 


True

In [277]:
from pydantic import ValidationError, field_validator
from pydantic.dataclasses import dataclass as p_dataclass

@p_dataclass
class Rectangle:
    length: float
    width: float

    @field_validator('length', 'width')
    @classmethod
    def validate_positive(cls, value: float):
        if value <= 0:
            raise ValueError("must be positive")
    




check("Rectangle(4, 5)", Rectangle);
check("Rectangle('wide', 'tall')", ValidationError);
check("Rectangle(-2, 2)", ValidationError, "positive");

✅ Rectangle(4, 5) -> Rectangle(length=None, width=None) 
✅ Rectangle('wide', 'tall') -> ValidationError 
✅ Rectangle(-2, 2) -> ValidationError 


In [280]:
from pydantic import ValidationError
from pydantic.dataclasses import dataclass as p_dataclass

@p_dataclass
class Person:
    name: str
    age: int

    @field_validator('age')
    @classmethod
    def validate_age_is_positive(cls, value: int):
        if value < 0:
            raise ValueError("must be positive.")
        
    @field_validator('name')
    @classmethod
    def validate_name_not_empty(cls, value: str):
        if not value:
            raise ValueError("may not be empty.")


check("Person('Nick', 37)", Person);
check("Person('Santa', 'old')", ValidationError, "integer");
check("Person('', -200)", ValidationError, "positive");
check("Person('', 12)", ValidationError, "empty");

✅ Person('Nick', 37) -> Person(name=None, age=None) 
✅ Person('Santa', 'old') -> ValidationError 
✅ Person('', -200) -> ValidationError 
✅ Person('', 12) -> ValidationError 
